In [8]:
import wikipedia
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.output_parsers import StrOutputParser

wikipedia.set_lang("pt")

@tool
def search_wikipedia(query: str) -> str:
    """
    Search Wikipedia in Portuguese for the given query and return the summary of the first result.
    
    Args:
        query (str): The search term to look up on Wikipedia.
    
    Returns:
        str: A summary of the first search result from Wikipedia.
    """
    titulo_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulo_paginas[:3]:  # Limiting to the first 3 results
        try:
            pagina = wikipedia.page(titulo, auto_suggest=True)
            resumos.append(f"{titulo}: {pagina.summary}")
        except wikipedia.exceptions.DisambiguationError as e:
            # If there's a disambiguation page, we can skip it or handle it differently
            continue
        except wikipedia.exceptions.PageError as e:
            # If the page does not exist, we can skip it
            continue
    if not resumos:
        return "Nenhum resultado encontrado."
    return "\n".join(resumos)

search_wikipedia.invoke({"query": "Python (programming language)"}) # type: ignore
function = convert_to_openai_function(search_wikipedia)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente que sabe sobre futebol e fala com sotaque nordestino"),
        ("user", "{query}"),
    ]
)

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = prompt | model.bind(functions=[function], function_call="auto") | StrOutputParser()
response = chain.invoke({"query": "Quem foi melhor Ronaldinho Gaúcho ou Messi? Em números"})  
print(response)

Bom, meu amigo, tanto Ronaldinho Gaúcho quanto Messi foram grandes jogadores, cada um com seu estilo e suas qualidades. Vamos dar uma olhada nos números para tentar te ajudar nessa comparação. Vou fazer uma pesquisa rápida para trazer algumas informações sobre os dois. Só um instante, vou retornar com os dados.
